## Install Dependencies

In [1]:
# https://github.com/jianfch/stable-ts
# You may have to do `!pip install certifi` and run the certificates in your application/python folder.
!pip install -U openai-whisper
!pip install -U git+https://github.com/jianfch/stable-ts.git
!pip install moviepy opencv-python Pi


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
  Cloning https://github.com/jianfch/stable-ts.git to /private/var/folders/11/j952l1hd3t7__pfbt3y3kk8c0000gn/T/pip-req-build-gu8h240o
  Running command git clone --filter=blob:none --quiet https://github.com/jianfch/stable-ts.git /private/var/folders/11/j952l1hd3t7__pfbt3y3kk8c0000gn/T/pip-req-build-gu8h240o
  Resolved https://github.com/jianfch/stable-ts.git to commit b336735ff784bb59690eec8f9f706b0151dda74c
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
  Using cached pi-0.1.2-py3-none-any.whl

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# https://github.com/openai/whisper/discussions/563
!brew uninstall librist --ignore-dependencies
!brew uninstall mbedtls --ignore-dependencies
!brew reinstall ffmpeg

Uninstalling /opt/homebrew/Cellar/librist/0.2.7_4... (28 files, 753.3KB)
Uninstalling /opt/homebrew/Cellar/mbedtls/3.5.1... (192 files, 12MB)
==> Downloading https://ghcr.io/v2/homebrew/core/ffmpeg/manifests/6.0_1-2
Already downloaded: /Users/richardawoyemi/Library/Caches/Homebrew/downloads/a839e675f8ecc4a6c3a689dae26e0deed55876713b87948b57ae6840dd6d5819--ffmpeg-6.0_1-2.bottle_manifest.json
==> Fetching dependencies for ffmpeg: mbedtls and librist
==> Downloading https://ghcr.io/v2/homebrew/core/mbedtls/manifests/3.5.1
Already downloaded: /Users/richardawoyemi/Library/Caches/Homebrew/downloads/b16d7db8b23100d49ccbf9a7ba52d8a989281b89810ae8211ba3ba5fe3199d0f--mbedtls-3.5.1.bottle_manifest.json
==> Fetching mbedtls
==> Downloading https://ghcr.io/v2/homebrew/core/mbedtls/blobs/sha256:737687d786
Already downloaded: /Users/richardawoyemi/Library/Caches/Homebrew/downloads/7c7059859732706191de6148e0cf3f7e06ab7b1672b3ffc0e37fc17e0dd40736--mbedtls--3.5.1.arm64_sonoma.bottle.tar.gz
==> Download

## Import Libraries

In [3]:
import stable_whisper
import whisper
import os
import cv2
from moviepy.editor import ImageSequenceClip, AudioFileClip, VideoFileClip
from tqdm import tqdm
from PIL import ImageFont, ImageDraw, Image
import shutil
import numpy as np

## Setup Initial Parameters

In [31]:
video_path = 'data/animating svgs.mp4'
output_video_path = 'data/captioned.mp4'
output_audio_path = 'data/extracted_audio.mp3'
frame_folder = "data/frames"
font_path = "font/Montserrat-Bold.ttf"
font_size = 40
text_pos_x = 0.5
text_pos_y = 0.5
text_colour = (255,255,255,255)
whisper_model = 'base'
char_per_line = 30  # Adjust this value based on the desired maximum characters per line
caption_lag = 0

## Extract Audio

In [5]:
video = VideoFileClip(video_path)
audio = video.audio 
audio.write_audiofile(output_audio_path)

MoviePy - Writing audio in data/extracted_audio.mp3


MoviePy - Done.


## Load Whisper Model

In [6]:
model = whisper.load_model('small')

In [7]:
st_model = stable_whisper.load_model('base')

## Transcribe Video

In [8]:
result = model.transcribe(output_audio_path)

In [9]:
result

{'text': " Hey guys, the best websites always have the most subtle animations and one of them that you see really often is animated SVG parts and the easiest way to do that is actually with Frame On Motion. Let me show you how. So I've got this simple Next.js project and first thing we're going to do is install Frame On Motion. Now I've got extra stuff in the project to make it beautiful but if we add in the SVG as you can see it's nice and static. Now animating this is simple. First import motion from Frame On Motion. Then we find the path tag and convert that to motions or path. So what we do is set the initial path length to be 0 and the animate path length to be 1 and we can then set the transition duration to be 0.5 and if we reload our webpage we'll see it's a working animation. Amazing. Guys, if you enjoyed this video give it a thumbs up, if you want to see more please subscribe and if you have any thoughts or questions give it in the comments below. Cheers.",
 'segments': [{'id

## Get Video

In [26]:
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
padded_width = width * 0.9
# asp = 16/9
# width = frame[:, int(int(width - 1 / asp * height) / 2):width - int((width - 1 / asp * height) / 2)].shape[1]

## Create Video Trasncript Array

In [27]:
text_array = []
for j in tqdm(result["segments"]):
    lines = []
    text = j["text"]
    end = j["end"]
    start = j["start"]
    total_frames = int((end - start) * fps)
    start = start * fps
    total_chars = len(text)    
    words = text.split(" ")
    current_line = words[0].strip()
    
    for i in range(1, len(words)):
        word = words[i].strip()
        if not word:
            continue
        
        # Check if adding the next word exceeds the character limit
        if len(current_line) + len(word) + 1 > char_per_line:
            lines.append([current_line, int(start) + caption_lag, int(len(current_line) / total_chars * total_frames) + int(start) + caption_lag])
            start = int(len(current_line) / total_chars * total_frames) + int(start)
            current_line = word
        else:
            current_line += " " + word
    
    # Append the last line
    lines.append([current_line.strip(), int(start) + caption_lag, int(len(current_line) / total_chars * total_frames) + int(start) + caption_lag])
    
    text_array.extend(lines)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 118520.36it/s]


In [28]:
text_array #A good time to update if there's any issues with the subtitle content

[[' Hey guys, the best websites', 0, 80],
 ['always have the most subtle', 80, 157],
 ['animations and one of them', 157, 232],
 ['that', 232, 243],
 [' you see really often is', 254, 318],
 ['animated SVG parts and the', 318, 388],
 ['easiest way to do that is', 388, 455],
 ['actually', 455, 476],
 ['with Frame On Motion.', 486, 536],
 ['Let me show you how.', 536, 586],
 [" So I've got this simple", 586, 642],
 ['Next.js project and first', 642, 701],
 ["thing we're going to do is", 701, 762],
 ['install', 762, 778],
 ['Frame On Motion.', 788, 838],
 [" Now I've got extra stuff in", 838, 933],
 ['the project to make it', 933, 1007],
 ['beautiful but if we add in the', 1007, 1109],
 ['SVG', 1109, 1119],
 [" as you can see it's nice and", 1132, 1266],
 ['static.', 1266, 1298],
 ['Now animating this is simple.', 1304, 1394],
 [' First import motion from', 1394, 1491],
 ['Frame On Motion.', 1491, 1553],
 [' Then we find the path tag and', 1557, 1668],
 ['convert that to motions or', 1668

## Generate Video Frames

In [29]:
def create_frame(input_frame, text, new_image_path):
    # Convert the cv2 frame to a PIL Image
    image = Image.fromarray(cv2.cvtColor(input_frame, cv2.COLOR_BGR2RGB))

    # Create a drawing object
    draw = ImageDraw.Draw(image)

    # Load the font
    font = ImageFont.truetype(font_path, font_size)

    # Calculate text position
    out = draw.textbbox((0, 0), text, font=font)
    x = int((width - out[2]) * text_pos_x)
    y = int(height * text_pos_y)

    # Draw the text on the image
    draw.text((x, y), text, font=font, fill=text_colour, stroke_width=2, stroke_fill="black")

    # Convert the PIL Image back to a cv2 frame
    image.save(new_image_path, overwrite=True)

In [30]:
if os.path.exists(frame_folder) and os.path.isdir(frame_folder):
    shutil.rmtree(frame_folder)

if not os.path.exists(frame_folder):
    os.makedirs(frame_folder)

N_frames = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    # frame = frame[:, int(int(width - 1 / asp * height) / 2):width - int((width - 1 / asp * height) / 2)]

    sub_applied = False
    
    for i in text_array:
        if N_frames >= i[1] and N_frames <= i[2]: # fix for frames that don't have any applicable subs
            create_frame(frame, i[0], os.path.join(frame_folder, str(N_frames) + ".jpg"))
            sub_applied = True
            break

    if not sub_applied:
        create_frame(frame, "", os.path.join(frame_folder, str(N_frames) + ".jpg"))

    N_frames += 1

cap.release()

Moviepy - Building video data/captioned.mp4.
MoviePy - Writing audio in captionedTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video data/captioned.mp4



Moviepy - Done !
Moviepy - video ready data/captioned.mp4


## Recreate Video

In [ ]:
images = [img for img in os.listdir(frame_folder) if img.endswith(".jpg")]
images.sort(key=lambda x: int(x.split(".")[0]))

frame = cv2.imread(os.path.join(frame_folder, images[0]))
height, width, layers = frame.shape

clip = ImageSequenceClip([os.path.join(frame_folder, image) for image in images], fps=fps)
audio = AudioFileClip(output_audio_path)
clip = clip.set_audio(audio)
clip.write_videofile(output_video_path)